In [1]:
#Импорт библиотек
import os
import mat_properties as prop
import numpy as n
import pandas as pd
import cotel
import nasos
import time
from scipy.optimize import root

#Время начала расчёта
start_time = time.time()

#Основные константы
calctolerance=10**-3
calcmethod = "hybr"
KPDnasos = 0.9
KPD=0.99

#таблица номинального режима
gas_streams0   = pd.read_excel('streams0.xlsx', sheet_name='gas',  index_col=0)
water_streams0 = pd.read_excel('streams0.xlsx', sheet_name='water',index_col=0)
#таблица задаваемых значений (заменится на интерфейс)
gas_streamsZAD    = pd.read_excel('streams.xlsx',  sheet_name='gas',  index_col=0)
water_streamsZAD  = pd.read_excel('streams.xlsx',  sheet_name='water',index_col=0)
#рабочая таблица (=номинал в 1 итерации)
gas_streams = gas_streams0.copy()
water_streams = water_streams0.copy()

#чтение с вводного экселя входных данных
Tgas = gas_streamsZAD.loc['GTU-PEVD','T']
Pgas = gas_streamsZAD.loc['GTU-PEVD','P']
Ggas = gas_streamsZAD.loc['GTU-PEVD','G']
Propgas = gas_streamsZAD.loc['GTU-PEVD','N2':'Ar']
Pvd = water_streamsZAD.loc['IVD-PEVD','P']
Pnd = water_streamsZAD.loc['IND-PPND','P']
Twat0 = water_streamsZAD.loc['X-GPK','T']

#запись состава газов в раб табл
gas_streams['N2'] = Propgas[0]
gas_streams['O2'] = Propgas[1]
gas_streams['CO2'] = Propgas[2]
gas_streams['H2O'] = Propgas[3]
gas_streams['Ar'] = Propgas[4]

#Состав газов в номинальном режиме и в частичной нагрузке
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
fractiongas0 = (gas_streams0.at['GTU-PEVD','N2'],gas_streams0.at['GTU-PEVD','O2'],gas_streams0.at['GTU-PEVD','CO2'],gas_streams0.at['GTU-PEVD','H2O'],gas_streams0.at['GTU-PEVD','Ar'])
fractiongas  = (gas_streams.at['GTU-PEVD','N2'],gas_streams.at['GTU-PEVD','O2'],gas_streams.at['GTU-PEVD','CO2'],gas_streams.at['GTU-PEVD','H2O'],gas_streams.at['GTU-PEVD','Ar'])

#Считывание рефпропа
RP = prop.init_REFPROP(r'C:\Program Files (x86)\REFPROP')
os.environ['RPPREFIX'] = r'C:/Program Files (x86)/REFPROP'
gas1 = prop.Materials_prop(gasmix,fractiongas, prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)
gas0 = prop.Materials_prop(gasmix,fractiongas0, prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)
water=prop.Materials_prop('water',[1.0,0,0,0,0], prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)

#Задание в раб табл параметров газа (1 строка)
gas_streams.loc['GTU-PEVD','T'] = Tgas
gas_streams['G'] = Ggas
gas_streams['P'] = Pgas
gas_streams.loc['GTU-PEVD','H'] = gas1.p_t(Pgas,Tgas)['h']

#Задание в раб табл параметров по воде
water_streams.loc['IVD-PEVD':'PEN-EVD','P'] = Pvd
water_streams.loc['IND-PPND':'X-GPK','P'] = Pnd
water_streams.loc['BND-PEN','P'] = Pnd
water_streams.loc['X-GPK','T'] = Twat0

#Расчёт параметров на насыщении в испарительных поверхностях (в 1 итерации приближение: недогрев до н.п. на входе 3°С)
water_streams.loc['IVD-PEVD','T'] = water.p_q(Pvd,1)['T']
water_streams.loc['IVD-PEVD','H'] = water.p_q(Pvd,1)['h']
water_streams.loc['IND-PPND','T'] = water.p_q(Pnd,1)['T']
water_streams.loc['IND-PPND','H'] = water.p_q(Pnd,1)['h']
water_streams.loc['EVD-IVD','T'] = water.p_q(Pvd,1)['T']-3
water_streams.loc['GPK-IND','T'] = water.p_q(Pnd,1)['T']-3
water_streams.loc['EVD-IVD','H'] = water.p_t(Pvd,water_streams.loc['EVD-IVD','T'])['h']
water_streams.loc['GPK-IND','H'] = water.p_t(Pnd,water_streams.loc['GPK-IND','T'])['h']



gas_streams.loc['PPND-IND','T'] = gas_streams.loc['EVD-PPND','T'] -10
gas_streams.loc['PPND-IND','H'] = gas1.p_t(gas_streams.loc['PPND-IND','P'],gas_streams.loc['PPND-IND','T'])['h']



#Расчёт
it=60
for i in range(it):
    ######################################################################
    #Связвка высокого давления
    for j in range(it):
#         print(gas_streams)
#         print(water_streams)
        #Связка ПЕВД+ИВД
        for k in range(it):
            
            #Расчёт ПЕВД
            PEVD = cotel.heatex('GTU-PEVD','PEVD-IVD','IVD-PEVD','PEVD-DROS',       KPD,calctolerance,gas0,gas1,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
            gas_streams.loc['PEVD-IVD','T':'G'] = [PEVD[0],PEVD[1],PEVD[2],PEVD[3]]
            water_streams.loc['PEVD-DROS','T':'G'] = [PEVD[4],PEVD[5],PEVD[6],PEVD[7]]
            
            #Расчёт ИВД
            IVD  = cotel.vapor ('PEVD-IVD','IVD-EVD','EVD-IVD','IVD-PEVD',           KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
            gas_streams.loc['IVD-EVD','T':'G'] = [IVD[0],IVD[1],IVD[2],IVD[3]]
            water_streams.loc['IVD-PEVD','T':'G'] = [IVD[4],IVD[5],IVD[6],IVD[7]]
            
            #Переопределение расхода в ВД
            water_streams.loc['PEVD-DROS':'PEN-EVD','G'] = IVD[7]
            water_streams.loc['BND-PEN','G'] = IVD[7]
            
            #Баланс ПЕВД+ИВД
            Qgas = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['PEVD-IVD','H'])
            Qwat = water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['IVD-PEVD','H'])
            print('dQ/Q ПЕВД+ИВД',((Qgas-Qwat)/Qgas))
            if  abs((Qgas-Qwat)/Qgas) < calctolerance:
                break;
        
        #Расчёт ЭВД
        EVD = cotel.heatex('IVD-EVD','EVD-PPND','PEN-EVD','EVD-IVD',               KPD,calctolerance,gas0,gas1,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
        gas_streams.loc['EVD-PPND','T':'G'] = [EVD[0],EVD[1],EVD[2],EVD[3]]
        water_streams.loc['EVD-IVD','T':'G'] = [EVD[4],EVD[5],EVD[6],EVD[7]]
        
        #Баланс ПЕВ+ИВД+ЭВД
        Qgas1 = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['IVD-EVD','H'])
        Qwat1 = water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['EVD-IVD','H'])
        print('dQ/Q ПЕВД+ИВД+ЭВД',((Qgas1-Qwat1)/Qgas1))
        if  abs((Qgas1-Qwat1)/Qgas1) < calctolerance:
            break;
    print("ВД: --- %s сек. ---" % round((time.time() - start_time),1))
    ######################################################################
    

    
    if i==0:
        gas_streams.loc['PPND-IND','T'] = gas_streams.loc['EVD-PPND','T'] -10
        gas_streams.loc['PPND-IND','H'] = gas1.p_t(gas_streams.loc['PPND-IND','P'],gas_streams.loc['PPND-IND','T'])['h']
    
    ######################################################################
    #Связка низкого давления
    for j in range(it):
        
        #Связка ППНД+ИНД
        for k in range(it):
            
            #Расчёт ППНД
            PPND = cotel.heatex('EVD-PPND','PPND-IND','IND-PPND','PPND-TURB',    KPD,calctolerance,gas0,gas1,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
            gas_streams.loc['PPND-IND','T':'G']    = [PPND[0],PPND[1],PPND[2],PPND[3]]
            water_streams.loc['PPND-TURB','T':'G'] = [PPND[4],PPND[5],PPND[6],PPND[7]]
            
            #Расчёт ИНД
            IND  = cotel.vaporND('PPND-IND','IND-GPK','GPK-IND','IND-PPND',      KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
            gas_streams.loc['IND-GPK','T':'G']    = [IND[0],IND[1],IND[2],IND[3]]
            water_streams.loc['IND-PPND','T':'G'] = [IND[4],IND[5],IND[6],IND[7]]
            
            #Переопределение расхода в НД
            water_streams.loc['PPND-TURB':'IND-PPND','G'] = IND[7]
            
            #ПЭН
            water_streams.loc['BND-PEN','T':'G'] = [IND[9],IND[10],IND[11],IND[12]]
            PEN=nasos.nasos('BND-PEN','PEN-EVD',water,KPDnasos,water_streams).calc()
            water_streams.loc['PEN-EVD','T':'G'] = [PEN[0],PEN[1],PEN[2],PEN[3]]
            print([IND[9],IND[10],IND[11],IND[12]])
            print([PEN[0],PEN[1],PEN[2],PEN[3]])
#             print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
#             print(gas_streams)
#             print(water_streams)
            
            
            
            #Баланс ППНД+ИНД
            Qgas = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['IND-GPK','H'])
            Qwat = water_streams.at['IND-PPND','G']*(water_streams.at['IND-PPND','H']-water_streams.at['GPK-IND','H'])+water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['IND-PPND','H'])+water_streams.at['BND-PEN','G']*(water_streams.at['BND-PEN','H']-water_streams.at['GPK-IND','H'])
            print('dQ/Q ППНД+ИНД',((Qgas-Qwat)/Qgas))
            if  abs((Qgas-Qwat)/Qgas) < calctolerance:
                break;

        #Расчёт расхода в ГПК
        water_streams.loc['X-GPK','G'] = water_streams.at['PPND-TURB','G']+water_streams.at['PEVD-DROS','G']
        water_streams.loc['GPK-IND','G'] = water_streams.at['X-GPK','G']
        
        #Расчёт ГПК
        GPK = cotel.heatex('IND-GPK','GPK-out','X-GPK','GPK-IND',           KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
        gas_streams.loc['GPK-out','T':'G']   = [GPK[0],GPK[1],GPK[2],GPK[3]]
        water_streams.loc['GPK-IND','T':'G'] = [GPK[4],GPK[5],GPK[6],GPK[7]]
        
        #Баланс ППНД+ИНД+ГПК
        Qgas1 = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['GPK-out','H'])
        Qwat1 = water_streams.at['GPK-IND','G']*(water_streams.at['GPK-IND','H']-water_streams.at['X-GPK','H'])+water_streams.at['IND-PPND','G']*(water_streams.at['IND-PPND','H']-water_streams.at['GPK-IND','H'])+water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['IND-PPND','H'])+water_streams.at['BND-PEN','G']*(water_streams.at['BND-PEN','H']-water_streams.at['GPK-IND','H'])
        print('dQ/Q ППНД+ИНД+ГПК',(Qgas1-Qwat1)/Qgas1)
        if  abs((Qgas1-Qwat1)/Qgas1) < calctolerance:
            break;
    print("НД+ --- %s сек. ---" % round((time.time() - start_time),1))
    ######################################################################

    #Баланс общий
    Qgasall = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['GPK-out','H'])
    Qwatall = water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['X-GPK','H'])+water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['X-GPK','H'])-water_streams.at['BND-PEN','G']*(water_streams.at['PEN-EVD','H']-water_streams.at['BND-PEN','H'])
    print('dQ/Qsumm',(Qgasall-Qwatall)/Qgasall)
    if  abs((Qgasall-Qwatall)/Qgasall) < calctolerance:
        break;

print('Степень сухости пара в ЭВД: ',water.p_h(water_streams.at['EVD-IVD','P'],water_streams.at['EVD-IVD','H'])['Q'])
print('Степень сухости пара в ГПК: ',water.p_h(water_streams.at['GPK-IND','P'],water_streams.at['GPK-IND','H'])['Q'])
print("fin:--- %s сек. ---" % round((time.time() - start_time),1))

dQ/Q ПЕВД+ИВД 0.5163542279017719
dQ/Q ПЕВД+ИВД -0.2904348273654389
dQ/Q ПЕВД+ИВД 0.07592024568374597
dQ/Q ПЕВД+ИВД -0.026344822931832646
dQ/Q ПЕВД+ИВД 0.008382885162416455
dQ/Q ПЕВД+ИВД -0.00274500551158718
dQ/Q ПЕВД+ИВД 0.0008905690800785455
dQ/Q ПЕВД+ИВД+ЭВД 0.1517279357142735
dQ/Q ПЕВД+ИВД -0.2657812431313067
dQ/Q ПЕВД+ИВД 0.07967829689678413
dQ/Q ПЕВД+ИВД -0.030893865482313675
dQ/Q ПЕВД+ИВД 0.010955566977601224
dQ/Q ПЕВД+ИВД -0.004015029063364797
dQ/Q ПЕВД+ИВД 0.0014540492472745223
dQ/Q ПЕВД+ИВД -0.0005288702437912647
dQ/Q ПЕВД+ИВД+ЭВД -0.06608326918930177
dQ/Q ПЕВД+ИВД 0.0927927448846283
dQ/Q ПЕВД+ИВД -0.0351225482788478
dQ/Q ПЕВД+ИВД 0.011959889092803124
dQ/Q ПЕВД+ИВД -0.004229148896409103
dQ/Q ПЕВД+ИВД 0.0014759786085252245
dQ/Q ПЕВД+ИВД -0.000517496605989125
dQ/Q ПЕВД+ИВД+ЭВД 0.027888510055137122
dQ/Q ПЕВД+ИВД -0.04394043490317998
dQ/Q ПЕВД+ИВД 0.015120049161071767
dQ/Q ПЕВД+ИВД -0.005452174467649292
dQ/Q ПЕВД+ИВД 0.0019337678327945322
dQ/Q ПЕВД+ИВД -0.0006899294023410008
dQ/Q 

In [2]:
#Вывод газовых потоков
gas_streams

,T,P,H,G,N2,O2,CO2,H2O,Ar
GTU-PEVD,544.500000,0.1,961.634975,317.6,0.7803,0.1237,0.0301,0.0594,0.0065
PEVD-IVD,462.519794,0.1,868.007532,317.6,0.7803,0.1237,0.0301,0.0594,0.0065
IVD-EVD,300.054794,0.1,687.697913,317.6,0.7803,0.1237,0.0301,0.0594,0.0065
EVD-PPND,198.303360,0.1,578.115933,317.6,0.7803,0.1237,0.0301,0.0594,0.0065
PPND-IND,218.502079,0.1,599.687110,317.6,0.7803,0.1237,0.0301,0.0594,0.0065
IND-GPK,156.176799,0.1,533.381265,317.6,0.7803,0.1237,0.0301,0.0594,0.0065
GPK-out,95.754131,0.1,469.730334,317.6,0.7803,0.1237,0.0301,0.0594,0.0065


In [3]:
#Вывод водяных потоков
water_streams

,T,P,H,G
PEVD-DROS,523.373398,5.47401,3484.078941,42.242463
IVD-PEVD,272.450253,5.71700,2787.569821,42.242463
EVD-IVD,272.450253,5.71700,1444.091466,42.242463
PEN-EVD,148.369709,5.71700,628.438156,42.242463
BND-PEN,147.656803,0.44700,622.076155,42.242463
PPND-TURB,147.338386,0.44315,2072.240303,10.117387
IND-PPND,147.656803,0.44700,2743.084030,10.117387
GPK-IND,147.656803,0.44700,633.976002,52.359850
X-GPK,60.000000,0.44700,251.748396,52.359850


In [4]:
#Невязка по всем параметрам
Qgas = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['PEVD-IVD','H'])
Qwat = water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['IVD-PEVD','H'])
print('dQ/Q ПЕВД+ИВД',((Qgas-Qwat)/Qgas))

Qgas1 = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['IVD-EVD','H'])
Qwat1 = water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['EVD-IVD','H'])
print('dQ/Q ПЕВД+ИВД+ЭВД',((Qgas1-Qwat1)/Qgas1))

Qgas = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['IND-GPK','H'])
Qwat = water_streams.at['IND-PPND','G']*(water_streams.at['IND-PPND','H']-water_streams.at['GPK-IND','H'])+water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['IND-PPND','H'])+water_streams.at['BND-PEN','G']*(water_streams.at['BND-PEN','H']-water_streams.at['GPK-IND','H'])
print('dQ/Q ППНД+ИНД',((Qgas-Qwat)/Qgas))

Qgas1 = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['GPK-out','H'])
Qwat1 = water_streams.at['GPK-IND','G']*(water_streams.at['GPK-IND','H']-water_streams.at['X-GPK','H'])+water_streams.at['IND-PPND','G']*(water_streams.at['IND-PPND','H']-water_streams.at['GPK-IND','H'])+water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['IND-PPND','H'])+water_streams.at['BND-PEN','G']*(water_streams.at['BND-PEN','H']-water_streams.at['GPK-IND','H'])
print('dQ/Q ППНД+ИНД+ГПК',(Qgas1-Qwat1)/Qgas1)

Qgasall = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['GPK-out','H'])
Qwatall = water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['X-GPK','H'])+water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['X-GPK','H'])-water_streams.at['BND-PEN','G']*(water_streams.at['PEN-EVD','H']-water_streams.at['BND-PEN','H'])
print('dQ/Qsumm',(Qgasall-Qwatall)/Qgasall)

dQ/Q ПЕВД+ИВД 0.0005589489813032808
dQ/Q ПЕВД+ИВД+ЭВД -0.0004842341666428516
dQ/Q ППНД+ИНД 0.0011983365853450647
dQ/Q ППНД+ИНД+ГПК 0.0004945969784565293
dQ/Qsumm -0.00016068662239445257
